In [2]:
from crawl_data import *
import pandas as pd
from datetime import date, datetime, timezone
import pytz

## 1. Dữ liệu danh sách các quỹ mở đang có


In [3]:
def unix_to_datetime_with_timezone(unix_timestamp):
    tz = pytz.timezone("Asia/Ho_Chi_Minh")
    if unix_timestamp:
        # Convert the Unix timestamp to a datetime object in UTC
        utc_dt = datetime.fromtimestamp(unix_timestamp / 1000, tz=timezone.utc)
        # Convert the datetime object to the specified timezone
        dt_with_timezone = utc_dt.astimezone(tz)
        return dt_with_timezone

In [4]:
df_funds = get_all_funds()
df_funds.head(2)

,id,name,shortName,code,price,nav,lastYearNav,holdingVolume,firstIssueAt,approveAt,description,createAt,updateAt,owner,status,dataFundAssetType
0,68,QUỸ ĐẦU TƯ CỔ PHIẾU KINH TẾ HIỆN ĐẠI VINACAPITAL,VMEEF,VMPF,10000.0,14945.25,11250.56,8554587.13,1681146000000,1698976912410,Quỹ VMEEF được đầu tư chủ yếu vào các cổ phiếu...,1698976594493,1723625818886,"{'id': 677, 'encodeURL': 'cong-ty-co-phan-quan...",PRODUCT_ACTIVE,"{'id': 1, 'name': 'Quỹ cổ phiếu', 'code': 'STO..."
1,11,QUỸ ĐẦU TƯ LỢI THẾ CẠNH TRANH BỀN VỮNG SSI,SSISCA,SSISCA,10000.0,39605.19,30573.98,3060194.03,1411664400000,1566827410082,Quỹ tăng trưởng NAV thông qua việc chủ động và...,1566826497938,1723626286873,"{'id': 606, 'encodeURL': 'cong-ty-tnhh-quan-ly...",PRODUCT_ACTIVE,"{'id': 1, 'name': 'Quỹ cổ phiếu', 'code': 'STO..."


In [5]:
# get list of funds and list of owner
list_fund_id = list(df_funds.id)
list_owner = list(df_funds.owner.drop_duplicates())
print(
    "Number of funds: {}, number of owners: {}".format(
        len(list_fund_id), len(list_owner)
    )
)

Number of funds: 50, number of owners: 22


In [6]:
# convert from unix to datetime
list_datetime_col = ["firstIssueAt", "approveAt", "updateAt", "createAt"]
for col in list_datetime_col:
    df_funds[col] = df_funds[col].apply(lambda x: unix_to_datetime_with_timezone(x))


# add owner_id
df_funds["owner_id"] = df_funds["owner"].apply(lambda x: x["id"])

# add fund type and code
df_funds["fund_type_name"] = df_funds["dataFundAssetType"].apply(lambda x: x["name"])
df_funds["fund_type_code"] = df_funds["dataFundAssetType"].apply(lambda x: x["code"])

df_funds = df_funds.drop(["dataFundAssetType", "owner"], axis=1)

In [7]:
df_funds.head(2)

,id,name,shortName,code,price,nav,lastYearNav,holdingVolume,firstIssueAt,approveAt,description,createAt,updateAt,status,owner_id,fund_type_name,fund_type_code
0,68,QUỸ ĐẦU TƯ CỔ PHIẾU KINH TẾ HIỆN ĐẠI VINACAPITAL,VMEEF,VMPF,10000.0,14945.25,11250.56,8554587.13,2023-04-11 00:00:00+07:00,2023-11-03 09:01:52.410000+07:00,Quỹ VMEEF được đầu tư chủ yếu vào các cổ phiếu...,2023-11-03 08:56:34.493000+07:00,2024-08-14 15:56:58.886000+07:00,PRODUCT_ACTIVE,677,Quỹ cổ phiếu,STOCK
1,11,QUỸ ĐẦU TƯ LỢI THẾ CẠNH TRANH BỀN VỮNG SSI,SSISCA,SSISCA,10000.0,39605.19,30573.98,3060194.03,2014-09-26 00:00:00+07:00,2019-08-26 20:50:10.082000+07:00,Quỹ tăng trưởng NAV thông qua việc chủ động và...,2019-08-26 20:34:57.938000+07:00,2024-08-14 16:04:46.873000+07:00,PRODUCT_ACTIVE,606,Quỹ cổ phiếu,STOCK


## 2. Danh sách các Tổ chức phát hành (owner)


In [8]:
df_owner = pd.DataFrame(list_owner)
df_owner.head(2)

,id,encodeURL,code,name,userId,userCode,email,email2,shortName,address1,...,website,templateContract,hsbcCode,securityDepositoryCenter,avatarUrl,isEnableEsign,isSignBeforeBuy,withdrawLimitSession,withdrawLimitDaily,buySellLimitDaily
0,677,cong-ty-co-phan-quan-ly-quy-vinacapital_677,007F02838278535,CÔNG TY CỔ PHẦN QUẢN LÝ QUỸ VINACAPITAL,677,007F02838278535,irwm@vinacapital.com,fincorp.vn@gmail.com,VINACAPITAL,"Lầu 17, 115 Nguyễn Huệ, Phường Bến Nghé, Quận ...",...,https://wm.vinacapital.com,https://files.fmarket.vn/pro/admin/68aa88fb-09...,VinaCapital,"{'id': 1, 'code': 'VSD', 'name': 'Trung tâm lư...",https://files.fmarket.vn/pro/temp/c7abba6d-aa5...,True,False,None,None,None
1,606,cong-ty-tnhh-quan-ly-quy-ssi_606,007F02838242897,CÔNG TY TNHH QUẢN LÝ QUỸ SSI,606,007F02838242897,info@ssi.com.vn,fincorp.vn@gmail.com,SSIAM,"Tầng 5, 1C Ngô Quyền, Phường Lý Thái Tổ, Quận ...",...,https://www.ssi.com.vn,https://files.fmarket.vn/pro/admin/82206c7c-e7...,ssiam,"{'id': 1, 'code': 'VSD', 'name': 'Trung tâm lư...",https://files.fmarket.vn/pro/temp/58175a8b-40c...,True,False,None,None,None


## 3. Danh sách top nắm giữ theo mã cổ phiếu/trái phiếu


In [27]:
top_holding = []
for fund_id in list_fund_id[:5]:
    top_holding.append(get_top_holding(fund_id))

# flatten list top hoding
top_holding = [stock_code for fund in top_holding for stock_code in fund]

In [32]:
df_top_holding = pd.DataFrame(top_holding)

df_top_holding["updateAt"] = df_top_holding["updateAt"].apply(lambda x: unix_to_datetime_with_timezone(x))
df_top_holding.head()

,id,stockCode,price,changeFromPrevious,changeFromPreviousPercent,industry,type,netAssetPercent,updateAt,fund_id
0,56777,FOX,93.2,1.90,2.08,Công nghệ và thông tin,STOCK,10.52,2024-08-08 06:00:16.489000+07:00,68
1,56791,VCB,91.7,0.20,0.22,Ngân hàng,STOCK,8.72,2024-08-08 06:00:16.489000+07:00,68
2,56771,ACB,24.8,0.15,0.61,Ngân hàng,STOCK,7.71,2024-08-08 06:00:16.489000+07:00,68
3,56778,FPT,134.8,1.10,0.82,Công nghệ và thông tin,STOCK,7.34,2024-08-08 06:00:16.489000+07:00,68
4,56772,BVH,45.5,0.15,0.33,Bảo hiểm,STOCK,4.56,2024-08-08 06:00:16.489000+07:00,68


## 4. Danh sách top nắm giữ theo loại tài sản

In [46]:
top_asset = []
for fund_id in list_fund_id[:5]:
    top_asset.append(get_asset_hoding(fund_id))

# flatten list top hoding
top_asset = [asset_code for asset in top_asset for asset_code in asset]

In [47]:
df_top_asset = pd.DataFrame(top_asset)

df_top_asset["updateAt"] = df_top_asset["updateAt"].apply(
    lambda x: unix_to_datetime_with_timezone(x)
)

df_top_asset["createAt"] = df_top_asset["createAt"].apply(
    lambda x: unix_to_datetime_with_timezone(x)
)

df_top_asset.head()

,id,assetType,assetPercent,createAt,updateAt,fund_id
0,1993,"{'id': 1, 'code': 'STOCK', 'name': 'Cổ phiếu',...",95.38,2024-08-08 06:00:16.519000+07:00,2024-08-08 06:00:16.519000+07:00,68
1,1994,"{'id': 3, 'code': 'CASH', 'name': 'Tiền và tươ...",4.62,2024-08-08 06:00:16.519000+07:00,2024-08-08 06:00:16.519000+07:00,68
2,2016,"{'id': 1, 'code': 'STOCK', 'name': 'Cổ phiếu',...",90.76,2024-08-08 06:00:25.101000+07:00,2024-08-08 06:00:25.101000+07:00,11
3,2017,"{'id': 3, 'code': 'CASH', 'name': 'Tiền và tươ...",9.24,2024-08-08 06:00:25.101000+07:00,2024-08-08 06:00:25.101000+07:00,11
4,2011,"{'id': 1, 'code': 'STOCK', 'name': 'Cổ phiếu',...",90.73,2024-08-08 06:00:23.885000+07:00,2024-08-08 06:00:23.885000+07:00,14


In [48]:
df_top_asset["assetName"] = df_top_asset["assetType"].apply(lambda x: x["name"])
df_top_asset["assetType"] = df_top_asset["assetType"].apply(lambda x: x["code"])

df_top_asset.head()

,id,assetType,assetPercent,createAt,updateAt,fund_id,assetName
0,1993,STOCK,95.38,2024-08-08 06:00:16.519000+07:00,2024-08-08 06:00:16.519000+07:00,68,Cổ phiếu
1,1994,CASH,4.62,2024-08-08 06:00:16.519000+07:00,2024-08-08 06:00:16.519000+07:00,68,Tiền và tương đương tiền
2,2016,STOCK,90.76,2024-08-08 06:00:25.101000+07:00,2024-08-08 06:00:25.101000+07:00,11,Cổ phiếu
3,2017,CASH,9.24,2024-08-08 06:00:25.101000+07:00,2024-08-08 06:00:25.101000+07:00,11,Tiền và tương đương tiền
4,2011,STOCK,90.73,2024-08-08 06:00:23.885000+07:00,2024-08-08 06:00:23.885000+07:00,14,Cổ phiếu


## 5. Danh sách top nắm giữ theo nhóm ngành

In [52]:
top_industry = []
for fund_id in list_fund_id[:5]:
    top_industry.append(get_industry_holding(fund_id))

# flatten list top hoding
top_industry = [industry for fund in top_industry for industry in fund]

In [54]:
df_top_industry = pd.DataFrame(top_industry)
df_top_industry.head()

,id,industry,assetPercent,fund_id
0,None,Ngân hàng,37.11,68
1,None,Công nghệ và thông tin,17.86,68
2,None,Xây dựng,6.33,68
3,None,Vận tải - Kho bãi,5.50,68
4,None,Bảo hiểm,4.56,68


## 5. Net asset value

In [80]:
start_date = date.today().strftime("%Y%m%d")
print(start_date)

20240830


In [81]:
list_nav = []
for fund_id in list_fund_id[:5]:
    list_nav.append(get_nav_history(fund_id))

# flatten list top hoding
list_nav = [nav for fund in list_nav for nav in fund]

df_nav = pd.DataFrame(list_nav)

df_nav.head()

,id,createdAt,nav,navDate,productId
0,21001,NaN,9996.94,2023-05-09,68
1,20963,NaN,10000.00,2023-05-11,68
2,20964,NaN,10085.36,2023-05-31,68
3,20965,NaN,10041.77,2023-06-01,68
4,20966,NaN,10084.79,2023-06-02,68


In [82]:
df_nav.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3626 entries, 0 to 3625
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         3626 non-null   int64  
 1   createdAt  2344 non-null   float64
 2   nav        3626 non-null   float64
 3   navDate    3626 non-null   object 
 4   productId  3626 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 141.8+ KB


In [83]:
df_nav["createdAt"][0] == False

np.False_

In [84]:
df_nav["navDate"] = pd.to_datetime(df_nav["navDate"])
df_nav["createdAt"] = df_nav["createdAt"].apply(
    lambda x: None if pd.isna(x) else unix_to_datetime_with_timezone(x)
)

df_nav.tail()

,id,createdAt,nav,navDate,productId
3621,29046,2024-08-15 16:49:17.356000+07:00,12151.85,2024-08-15,46
3622,29149,2024-08-20 16:37:15.864000+07:00,12544.62,2024-08-20,46
3623,29221,2024-08-22 16:29:28.474000+07:00,12654.76,2024-08-22,46
3624,30007,2024-08-27 16:30:26.010000+07:00,12575.45,2024-08-27,46
3625,30094,2024-08-30 09:11:38.706000+07:00,12537.37,2024-08-29,46


In [85]:
df_nav.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3626 entries, 0 to 3625
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype                           
---  ------     --------------  -----                           
 0   id         3626 non-null   int64                           
 1   createdAt  2344 non-null   datetime64[ns, Asia/Ho_Chi_Minh]
 2   nav        3626 non-null   float64                         
 3   navDate    3626 non-null   datetime64[ns]                  
 4   productId  3626 non-null   int64                           
dtypes: datetime64[ns, Asia/Ho_Chi_Minh](1), datetime64[ns](1), float64(1), int64(2)
memory usage: 141.8 KB
